In [1]:
import pandas as pd

In [4]:
df = pd.read_excel('QAES_trait_scores_with_final.xlsx')

In [5]:
df.head()

,essay_id,essay_set,topic,organization_r1,vocabulary_r1,style_r1,development_r1,mechanics_r1,structure_r1,relevance_r1,...,style_fn,development_fn,mechanics_fn,structure_fn,relevance_fn,final_score,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36
0,1,1,1: communication,4,4,4,4,4,4,2,...,4,4,4,4,2,26,NaN,NaN,NaN,NaN
1,3,1,1: communication,4,4,4,4,4,4,2,...,4,4,4,4,2,26,NaN,NaN,NaN,NaN
2,4,2,2: technology,5,4,4,4,4,4,2,...,4,4,4,4,2,27,NaN,NaN,NaN,NaN
3,5,2,2: technology,5,5,4,4,4,4,2,...,4,4,4,4,2,28,NaN,NaN,NaN,NaN
4,6,1,1: communication,3,3,2,3,2,3,1,...,2,3,2,3,1,17,NaN,NaN,NaN,NaN


In [19]:
df.columns

Index(['essay_id', 'essay_set', 'topic', 'organization_r1', 'vocabulary_r1',
       'style_r1', 'development_r1', 'mechanics_r1', 'structure_r1',
       'relevance_r1', 'organization_r2', 'vocabulary_r2', 'style_r2',
       'development_r2', 'mechanics_r2', 'structure_r2', 'relevance_r2',
       'r3_flag', 'organization_r3', 'vocabulary_r3', 'style_r3',
       'development_r3', 'mechanics_r3', 'structure_r3', 'relevance_r3',
       'organization_fn', 'vocabulary_fn', 'style_fn', 'development_fn',
       'mechanics_fn', 'structure_fn', 'relevance_fn', 'final_score',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36'],
      dtype='object')

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load Qwen 1.5B model
model_name = "Qwen/Qwen1.5-1.8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, trust_remote_code=True).cuda()
model.eval()

def ask_question(question: str):
    inputs = tokenizer(question, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [6]:
# Example usage
question = """
You are an expert Arabic language evaluator. Your task is to assess the proficiency of an Arabic essay based on seven traits:
1. Organization (0-5): How well-structured and coherent is the essay?
2. Vocabulary (0-5): Does the writer use a rich and appropriate vocabulary?
3. Style (0-5): Is the writing engaging, fluent, and stylistically appropriate?
4. Development (0-5): Are ideas elaborated with sufficient details and examples?
5. Mechanics (0-5): Are grammar, spelling, and punctuation correct?
6. Structure (0-5): Does the essay follow proper syntactic structures?
7. Relevance (0-2): Does the essay address the given topic appropriately?

Each trait should be scored on a scale from 0 (poor) to 5 (excellent).
Finally, calculate the total score by summing all traits, with a maximum possible score of 32.

Return ONLY this JSON object with your scores (replace X with actual numbers):
{
    "organization": X,
    "vocabulary": X,
    "style": X,
    "development": X,
    "mechanics": X,
    "structure": X,
    "relevance": X,
    "final_score": X
}


Essay:
إن الثورة التكنولوجية تعد السمة التي تصبغ عصرنا الحالي، وهي علامة فارقة تميز بين الأجيال. ويعد استخدام الهواتف الجوالة والبريد الإلكتروني أحد أهم مظاهر الثورة التكنولوجية والتطور في عمليات التواصل في الوقت الحالي.
وكأي ظاهرة تطفو على السطح في المجتمع، فهناك إيجابيات وسلبيات، وهناك المعارضون والمُؤيدون ولكل منهم أسبابه ومبرراته التي يؤيد بها رأيه.
إن استخدام الهواتف الجوالة والبريد الإلكتروني قدم للإنسانية بشكل عام فرص التواصل السريع، وجعل العالم قرية واحدة، وتخطى بذلك الحدود الزمنية والجغرافية، فيمكنك التواصل مع من تحب بأي بقعة بالأرض ويمكنك الوصول إليه بأي وقت صوتا وصورة، فهناك العديد من التطبيقات التي يتم تحميلها على الهواتف الجوالة تقدم خدمة التواصل الصوتي والفيديو وبدون تكاليف إضافية. فعلى سبيل المثال: العائلات التي كان لها أبناء يدرسون أو يعملون بالخارج، كانت عملية التواصل صعبة وتستغرق وقتا طويلا، فكان يتم الاعتماد على الرسائل البريدية التي قد يستلمها المستلم بعد عدة أشهر. لقد كانت الغربة عن الأهل والأصدقاء صعبة نتيجة لقلة التواصل، ولكن بعد استخدام الهواتف الجوالة بتطبيقاتها العديدة، أصبح التواصل متاحا في أي وقت وبالتالي لا يشعر الإنسان بالغربة أثناء السفر للعمل أو للدراسة أو حتى للسياحة.
كما أن هذه الوسائل التكنولوجية سهلت من عملية إرسال الملفات وبأحجام كبيرة عن طريق إرفاقها بالبريد الإلكتروني ويستلمها المستلم بمجرد ضغطة واحدة (إرسال). إن مقارنة هذا الوضع الحالي من التطور في مجال التواصل مع ما كان سابقا يجعل المرء يتساءل: كيف كان يعيش العالم من قبل بدون الهواتف الجوالة والبريد الإلكتروني؟ إنها حقا معاناة!
وكأي أمر في هذه الحياة، فإن لكل شيء ثمن، ولكن نتساءل ما الثمن الذي دفعته البشرية مقابل هذا التواصل الذي تخطى الحدود الزمنية والجغرافية؟
إن اعتماد البشرية على وسائل التكنولوجيا في التواصل تولدت عنه العديد من السلبيات التي طالت العلاقات الاجتماعية بين الناس. فنلاحظ أن الزيارات الاجتماعية قد تقلصت نتيجة لذلك، مما أثر بشكل ملحوظ على المهارات والجوانب الاجتماعية لدى الناس. فأصبحت ترسل التهاني بالأعياد والمناسبات عن طريق التطبيقات المختلفة مثل رسائل الواتس آب، السناب شات، الفيسبوك، وغيرها. مما قلص من التواصل الاجتماعي الفعلي بين الناس مما أدى إلى ضعف العلاقات الاجتماعية التي كانت ميزة تميز عصر ما قبل التكنولوجيا.

وقد امتد هذا الأثر السلبي فطال تفضيلات الإنسان، فحتى عندما يقوم بزيارة اجتماعية نجده منشغلا بهاتفه الجوال مما جعل من بعض كبار السن أن يطلبوا من أبنائهم وأحفادهم ترك الجوالات في سلة توضع خصيصا عند مدخل البيت، حتى لا ينشغلوا بها خلال الاجتماعات الأسرية والاجتماعية.

وفيما يتعلق بالبريد الإلكتروني، فنلاحظ أنه يستخدم عادة في مجال العمل، فقد سهل عملية إرسال التعليمات والتواصل بين مجموعة من الناس يعملون في مجال واحد. ولكن في نفس الوقت أصبحت هذه الميزة نقمة حيث لم يعد هناك وقتا خاصا خارج أوقات العمل، ففكرة إمكانية الوصول إليك في أي وقت لها سلبياتها وإيجابياتها. ففي السابق، عندما ينتهي الإنسان من عمله ويعود للمنزل، لا يفكر إلا في الأسرة والحياة الاجتماعية، ولكن بعد الجوال والبريد الإلكتروني، أصبح هناك تعدي على هذه الخصوصية.

لقد أصبح لدى الفرد في وقتنا الحالي هوس الارتباط بالهاتف الجوال، فلا يمكنه أن يفارقه، ومن وقت لآخر، يراجع مع تم استلامه من الأصدقاء أو الأهل أو المدرسين أو مكان العمل، مما أدى إلى عدم وجود الخصوصية وتعرض الفرد للضغوط نتيجة هذا الارتباط بالهاتف الجوال.

وفي نهاية الأمر، يمكننا القول أن إيجابيات وسلبيات التكنولوجيا بشكل عام واضحة وجلية للجميع، والتكنولوجيا شأنها شأن أي شيء هو سلاح ذو حدين، وعلى الفرد أن يكون واعيا أثناء استخدامها بالسلبيات والإيجابيات، وبناء عليه يستفيد من التكنولوجيا ويستخدمها بشكل مقنن لأننا لا نستطيع أن نتجاهل العصر ونرفض التطور، لأن التطور هو سمة الحياة.
"""
answer = ask_question(question)

print("Answer:")

print(answer)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Answer:

You are an expert Arabic language evaluator. Your task is to assess the proficiency of an Arabic essay based on seven traits:
1. Organization (0-5): How well-structured and coherent is the essay?
2. Vocabulary (0-5): Does the writer use a rich and appropriate vocabulary?
3. Style (0-5): Is the writing engaging, fluent, and stylistically appropriate?
4. Development (0-5): Are ideas elaborated with sufficient details and examples?
5. Mechanics (0-5): Are grammar, spelling, and punctuation correct?
6. Structure (0-5): Does the essay follow proper syntactic structures?
7. Relevance (0-2): Does the essay address the given topic appropriately?

Each trait should be scored on a scale from 0 (poor) to 5 (excellent).
Finally, calculate the total score by summing all traits, with a maximum possible score of 32.

Return ONLY this JSON object with your scores (replace X with actual numbers):
{
    "organization": X,
    "vocabulary": X,
    "style": X,
    "development": X,
    "mechanics